In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import guidance
from textwrap import dedent
import json 

from grammar_guide import guide, load_parser

In [11]:
model_name_or_path = "HuggingFaceTB/SmolLM-135M"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
parser = load_parser(lark_grammar_filepath="../grammars/json.lark")

In [7]:
def generate(text: str):
    return """{"name": "John"""

In [8]:
res = guide(
    model=generate,
    parser=parser,
    prompt=dedent("""
    Here is a really long, nested JSON that extracts fields from this sentence:\n\nMy name is Joseph Smith, and I work at Apple. I'm 32 years old, and my interests include kayaking, skiing, snowboarding, and woodworking.\n\n```json\n
    """),
)

Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Cannot find a valid prediction after 3 retries


In [14]:
res = guide(
    model,
    tokenizer=tokenizer,
    parser=parser,
    seed_str="""{"name":""",
    prompt=dedent("""
    Here is a really long, nested JSO that extracts fields from this sentence:\n\nMy name is Joseph Smith, and I work at Apple. I'm 32 years old, and my interests include kayaking, skiing, snowboarding, and woodworking.\n\n```json\n
    """),
    draft_model=guidance.models.Transformers(
        model_name_or_path, echo=False
    ),
    stop_at=['```'],
    max_grammar_corrections=10,
    max_new_tokens=6,
    temperature=0.3,
)
res.process_time_seconds

Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...


Here is a really long , nested J SO that extracts fields from this sentence : 
 
 My name is Joseph Smith , and I work at Apple . I 'm 3 2 years old , and my interests include kay aking , skiing , snow boarding , and woodworking . 
 
 ``` json 

 {" name ": ":" Joseph Smith , 3 2 " ", " age ": 3 2 , " , " job ":" K ay ak " ", " location ":" New York "} ," job ":" K ay ak ", " location ":" New York " , ", " occupation ":" Sk i ", " ", " phone ":" 1 2 3 " ", " address ":" 1 3 0

2.3670783042907715

In [15]:
try:
    print(json.dumps(json.loads(res.response), indent = 4))
except:
    print(res.response)

{
    "name": "Joseph Smith, 32",
    "age": 32,
    "job": "Kayak",
    "location": "New York",
    "occupation": "Ski",
    "phone": "123"
}
